이를 위해 pyspark과 Py4J 패키지를 설치한다. Py4J 패키지는 파이썬 프로그램이 자바가상머신상의 오브젝트들을 접근할 수 있게 해준다. Local Standalone Spark을 사용한다.

In [1]:
!pip install pyspark==3.3.1 py4j==0.10.9.5

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 281.4/281.4 MB 3.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.7/199.7 kB 11.2 MB/s eta 0:00:00
  Created wheel for pyspark: filename=pyspark-3.3.1-py2.py3-none-any.whl size=281845493 sha256=b7763c0848307b595245d541221e36a57ddfe068bd600de6e7661969968f1e38
  Stored in directory: /root/.cache/pip/wheels/0f/f0/3d/517368b8ce80486e84f89f214e0a022554e4ee64969f46279b
Successfully built pyspark
  Attempting uninstall: py4j
    Found existing installation: py4j 0.10.9.7
    Uninstalling py4j-0.10.9.7:
      Successfully uninstalled py4j-0.10.9.7


In [2]:
!wget https://s3-geospatial.s3.us-west-2.amazonaws.com/orders.csv

--2024-06-19 06:00:01--  https://s3-geospatial.s3.us-west-2.amazonaws.com/orders.csv
Resolving s3-geospatial.s3.us-west-2.amazonaws.com (s3-geospatial.s3.us-west-2.amazonaws.com)... 52.218.133.18, 52.92.147.90, 52.92.160.138, ...
Connecting to s3-geospatial.s3.us-west-2.amazonaws.com (s3-geospatial.s3.us-west-2.amazonaws.com)|52.218.133.18|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 89951 (88K) [text/csv]
Saving to: ‘orders.csv’

orders.csv          100%[===================>]  87.84K  --.-KB/s    in 0.08s   

2024-06-19 06:00:01 (1.11 MB/s) - ‘orders.csv’ saved [89951/89951]



**Spark Session**

In [3]:
from pyspark.sql import SparkSession

spark = SparkSession \
    .builder \
    .appName("Python Spark Hive") \
    .enableHiveSupport() \
    .getOrCreate()

In [4]:
!ls -tl

total 92
drwxr-xr-x 1 root root  4096 Jun 14 17:39 sample_data
-rw-r--r-- 1 root root 89951 Apr 24  2022 orders.csv


In [5]:
# Read in data and create a DataFrame
df = spark.read.csv("orders.csv", inferSchema=True, header=True, sep ='\t')

In [6]:
df.show(5)

+------------+--------------------+
|    order_id|               items|
+------------+--------------------+
|860196503764|[{"name": "DAILY ...|
|860292645076|[{"name": "DAILY ...|
|860320956628|[{"name": "DAILY ...|
|860321513684|[{"name": "DAILY ...|
|862930665684|[{"name": "DAILY ...|
+------------+--------------------+
only showing top 5 rows



In [8]:
spark.sql("CREATE DATABASE IF NOT EXISTS TEST_DB")
spark.sql("USE TEST_DB")

DataFrame[]

In [9]:
spark.sql("SHOW DATABASES").show()

+---------+
|namespace|
+---------+
|  default|
|  test_db|
+---------+



In [10]:
!ls -tl

total 104
drwxr-xr-x 3 root root  4096 Jun 19 06:01 spark-warehouse
drwxr-xr-x 5 root root  4096 Jun 19 06:00 metastore_db
-rw-r--r-- 1 root root   671 Jun 19 06:00 derby.log
drwxr-xr-x 1 root root  4096 Jun 14 17:39 sample_data
-rw-r--r-- 1 root root 89951 Apr 24  2022 orders.csv


In [11]:
df.write.saveAsTable("TEST_DB.orders", mode="overwrite")

In [12]:
!ls -tl spark-warehouse/test_db.db/orders/

total 24
-rw-r--r-- 1 root root     0 Jun 19 06:01 _SUCCESS
-rw-r--r-- 1 root root 23051 Jun 19 06:01 part-00000-4c05ce20-3172-4f7a-ab9e-0cb0b8bc1d8e-c000.snappy.parquet


In [13]:
spark.sql("SELECT * FROM TEST_DB.orders").show(5)

+------------+--------------------+
|    order_id|               items|
+------------+--------------------+
|860196503764|[{"name": "DAILY ...|
|860292645076|[{"name": "DAILY ...|
|860320956628|[{"name": "DAILY ...|
|860321513684|[{"name": "DAILY ...|
|862930665684|[{"name": "DAILY ...|
+------------+--------------------+
only showing top 5 rows



In [14]:
spark.table("TEST_DB.orders").show(5)

+------------+--------------------+
|    order_id|               items|
+------------+--------------------+
|860196503764|[{"name": "DAILY ...|
|860292645076|[{"name": "DAILY ...|
|860320956628|[{"name": "DAILY ...|
|860321513684|[{"name": "DAILY ...|
|862930665684|[{"name": "DAILY ...|
+------------+--------------------+
only showing top 5 rows



In [15]:
!ls -tl metastore_db/

total 28
drwxr-xr-x 2 root root 4096 Jun 19 06:01 seg0
-rw-r--r-- 1 root root  900 Jun 19 06:00 service.properties
-rw-r--r-- 1 root root  608 Jun 19 06:00 README_DO_NOT_TOUCH_FILES.txt
drwxr-xr-x 2 root root 4096 Jun 19 06:00 log
-rw-r--r-- 1 root root   38 Jun 19 06:00 db.lck
-rw-r--r-- 1 root root    4 Jun 19 06:00 dbex.lck
drwxr-xr-x 2 root root 4096 Jun 19 06:00 tmp


In [16]:
spark.catalog.listTables()

[Table(name='orders', database='test_db', description=None, tableType='MANAGED', isTemporary=False)]

In [17]:
spark.sql("""
    DROP TABLE IF EXISTS TEST_DB.orders_count;
""")

DataFrame[]

In [18]:
spark.sql("""
    CREATE TABLE TEST_DB.orders_count AS
    SELECT order_id, COUNT(1) as count
    FROM TEST_DB.orders
    GROUP BY 1""")

DataFrame[]

In [19]:
spark.catalog.listTables()

[Table(name='orders', database='test_db', description=None, tableType='MANAGED', isTemporary=False),
 Table(name='orders_count', database='test_db', description=None, tableType='MANAGED', isTemporary=False)]

In [20]:
!ls -tl spark-warehouse/test_db.db/

total 8
drwxr-xr-x 2 root root 4096 Jun 19 06:03 orders_count
drwxr-xr-x 2 root root 4096 Jun 19 06:01 orders


In [21]:
spark.sql("SELECT * FROM test_db.orders_count LIMIT 10").show()

+-------------+-----+
|     order_id|count|
+-------------+-----+
|1209310019796|    1|
|1253925257428|    1|
|1314797846740|    1|
|1370637402324|    1|
|1967690285268|    1|
|1971226443988|    1|
|2106987970772|    1|
|2135281533140|    1|
|1271771070676|    1|
|1713331765460|    1|
+-------------+-----+

